In [2]:
import pandas as pd
import datetime as dt

In [3]:
metro_value = 'Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv'
metro_rent = 'Metro_ZORI_AllHomesPlusMultifamily_SSA.csv'

zip_value = 'Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv'
zip_rent = 'Zip_ZORI_AllHomesPlusMultifamily_SSA.csv'

In [52]:
home_value_zip = pd.read_csv(zip_value)
rent_value_zip = pd.read_csv(zip_rent)
zip_code_list = home_value_zip[['RegionName', 'City', 'State']]

In [8]:
target_years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020']
home_value_by_month = []
rent_value_by_month = []

In [9]:
for year in target_years:
    home_data_columns = ['RegionName']
    rent_data_columns = ['RegionName']
    for column in home_value_zip.columns:
        if year in column:
            home_data_columns.append(column)
        else:
            pass
    home_df = home_value_zip[home_data_columns]
    home_value_by_month.append(home_df)
    for column in rent_value_zip.columns:
        if year in column:
            rent_data_columns.append(column)
        else:
            pass
    rent_df = rent_value_zip[rent_data_columns]
    rent_value_by_month.append(rent_df)

In [27]:
for df in home_value_by_month:
    df['Annual Average'] = df.mean(axis=1)

for df in rent_value_by_month:
    df['Annual Average'] = df.mean(axis=1)

In [31]:
home_value_by_year = []
rent_value_by_year = []

for df in home_value_by_month:
    annual_summary = df[['RegionName', 'Annual Average']]
    annual_summary = pd.merge(annual_summary, zip_code_list, on='RegionName')
    annual_summary = annual_summary[['State', 'City', 'RegionName', 'Annual Average']]
    annual_summary = annual_summary.rename(columns={'RegionName': 'ZIP Code', 'Annual Average': 'Home Value'})
    home_value_by_year.append(annual_summary)

for df in rent_value_by_month:
    annual_summary = df[['RegionName', 'Annual Average']]
    annual_summary = pd.merge(annual_summary, zip_code_list, on='RegionName')
    annual_summary = annual_summary[['State', 'City', 'RegionName', 'Annual Average']]
    annual_summary = annual_summary.rename(columns={'RegionName': 'ZIP Code', 'Annual Average': 'Avg Rent'})
    rent_value_by_year.append(annual_summary)

In [59]:
combined_by_zip = []
combined_by_city = []
combined_by_state = []

for i in range(len(home_value_by_year)):
    combined = pd.merge(home_value_by_year[i], rent_value_by_year[i], on='ZIP Code', how='inner')
    combined['Ratio'] = combined['Avg Rent']/combined['Home Value']
    combined = combined[[
        'State_x',
        'City_x',
        'ZIP Code',
        'Home Value',
        'Avg Rent',
        'Ratio'
    ]]
    combined = combined.rename(columns={
        'State_x': 'State',
        'City_x': 'City',
    })
    combined_by_zip.append(combined)
    combined_city = combined[['State', 'City', 'Home Value', 'Avg Rent', 'Ratio']].groupby(['State', 'City']).mean().reset_index()
    combined_by_city.append(combined_city)
    combined_state = combined[['State', 'Home Value', 'Avg Rent', 'Ratio']].groupby('State').mean().reset_index()
    combined_by_state.append(combined_state)

In [67]:
print(len(combined_by_state))

7


In [71]:
yearly_ratios_zip = {
    'ZIP Code': combined_by_zip[0]['ZIP Code']
}
yearly_ratios_city = {
    'City': combined_by_city[0]['City']
}
yearly_ratios_state = pd.DataFrame({
    'State': combined_by_state[0]['State']
})

In [74]:
for i in range(len(combined_by_state)):
    yearly_ratios_state[f'New column {i}'] = combined_by_state[i]['Ratio']*100

In [75]:
print(yearly_ratios_state)

State  New column 0  New column 1  New column 2  New column 3  \
0     AL      3.919653      2.660687      2.617824      2.402707   
1     AR     16.924035     11.935846     14.635136     11.149871   
2     AZ      4.349514      4.032043      3.732092      3.497790   
3     CA      2.832775      2.268166      2.027819      1.818087   
4     CO      4.007253      2.974326      2.523895      2.293865   
5     CT      2.020598      0.995277      0.908034      0.876561   
6     DC      1.163809      0.841405      0.752543      0.704020   
7     DE      2.362287      2.021664      1.734852      1.627744   
8     FL      2.751715      2.609176      2.264490      2.035165   
9     GA      2.642904      2.385459      2.235084      2.017747   
10    HI      2.422482      1.813118      1.685042      1.567619   
11    IA      4.428873      3.754501      3.272560      3.054173   
12    ID      4.794926      4.030529      3.732955      3.409229   
13    IL      4.872494      4.112543      3.446000 